In [ ]:
# =============================================================================
# 02_data_preparation.ipynb
# =============================================================================
# FarmTech Solutions - Análise de Rendimento de Safra
# EDA Completa: Análise Exploratória dos Dados
# Versão: 1.0
# =============================================================================

## 📊 Objetivo da Preparação

Este notebook realiza a **Preparação de Dados** para modelagem preditiva.

### 🎯 Metas da Preparação:
- **Limpeza de Dados**: Tratamento de outliers e valores faltantes
- **Feature Engineering**: Criação de novas variáveis derivadas
- **Encoding**: Tratamento de variáveis categóricas
- **Normalização**: Padronização de variáveis numéricas

## 1. 🚀 Setup e Imports

### Bibliotecas Necessárias
Vamos importar as bibliotecas essenciais para preparação de dados:
- **pandas**: manipulação e análise de dados
- **numpy**: operações numéricas
- **scikit-learn**: preparação e transformação de dados
- **matplotlib & seaborn**: visualizações

In [3]:
# =============================================================================
# SETUP E IMPORTS
# =============================================================================

# Importar bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")

print("🔧 Setup da preparação de dados concluído!")
print("✅ Bibliotecas importadas com sucesso!")

🔧 Setup da preparação de dados concluído!
✅ Bibliotecas importadas com sucesso!


## 2.1 📥 Carregamento e Verificação Inicial

### Objetivo da Seção
Realizar verificação sistemática dos dados para identificar:
- **Integridade**: Valores faltantes, duplicatas e inconsistências
- **Qualidade**: Tipos de dados e ranges de valores
- **Consistência**: Valores dentro de limites esperados
- **Base**: Preparação para próximas etapas de limpeza

### Estratégia de Verificação:
- **Análise de valores faltantes**: Identificar e quantificar dados ausentes
- **Verificação de duplicatas**: Detectar observações repetidas
- **Análise de tipos**: Confirmar tipos corretos de dados
- **Verificação de ranges**: Validar limites mínimos e máximos

In [4]:
# =============================================================================
# CARREGAMENTO E VERIFICAÇÃO INICIAL DOS DADOS
# =============================================================================

print("📊 Carregando dados para preparação...")

# Carregar dados
df = pd.read_csv('../data/raw/crop_yield.csv')

print(f"✅ Dataset carregado: {df.shape[0]} registros, {df.shape[1]} variáveis")

# Verificação de integridade
print("\n🔍 VERIFICAÇÃO DE INTEGRIDADE:")
print(f"   → Valores faltantes: {df.isnull().sum().sum()}")
print(f"   → Duplicatas: {df.duplicated().sum()}")
print(f"   → Tipos de dados: {df.dtypes.to_dict()}")

# Verificação de consistência
print("\n📊 VERIFICAÇÃO DE CONSISTÊNCIA:")
for col in df.select_dtypes(include=[np.number]).columns:
    print(f"   → {col}: {df[col].min():.2f} a {df[col].max():.2f}")

print("✅ Verificação inicial concluída!")

📊 Carregando dados para preparação...
✅ Dataset carregado: 156 registros, 6 variáveis

🔍 VERIFICAÇÃO DE INTEGRIDADE:
   → Valores faltantes: 0
   → Duplicatas: 0
   → Tipos de dados: {'Crop': dtype('O'), 'Precipitation (mm day-1)': dtype('float64'), 'Specific Humidity at 2 Meters (g/kg)': dtype('float64'), 'Relative Humidity at 2 Meters (%)': dtype('float64'), 'Temperature at 2 Meters (C)': dtype('float64'), 'Yield': dtype('int64')}

📊 VERIFICAÇÃO DE CONSISTÊNCIA:
   → Precipitation (mm day-1): 1934.62 a 3085.79
   → Specific Humidity at 2 Meters (g/kg): 17.54 a 18.70
   → Relative Humidity at 2 Meters (%): 82.11 a 86.10
   → Temperature at 2 Meters (C): 25.56 a 26.81
   → Yield: 5249.00 a 203399.00
✅ Verificação inicial concluída!


## 2.2 🔍 Análise de Outliers

### Objetivo da Seção
Identificar e analisar valores extremos para decidir estratégia de tratamento:
- **Detecção**: Identificar outliers usando métodos estatísticos robustos
- **Análise**: Compreender origem e impacto dos valores extremos
- **Decisão**: Definir estratégia de tratamento (remover, transformar, manter)
- **Base**: EDA mostrou 12 outliers em Temperature e 35 em Yield

### Estratégia de Análise:
- **Método IQR**: Detecção baseada em quartis (Q1 - 1.5×IQR, Q3 + 1.5×IQR)
- **Método Z-Score**: Detecção baseada em desvios padrão (|z| > 3)
- **Análise contextual**: Considerar significado agrícola dos outliers
- **Decisão inteligente**: Não remover automaticamente, analisar contexto

In [5]:
# =============================================================================
# ANÁLISE DE OUTLIERS - DETECÇÃO E ANÁLISE
# =============================================================================

print("🔍 Iniciando análise de outliers...")

# Selecionar variáveis numéricas
numeric_vars = ['Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)',
                'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)', 'Yield']

# Método IQR para detecção de outliers
outliers_iqr = {}
outliers_zscore = {}

for var in numeric_vars:
    # Método IQR
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_iqr[var] = df[(df[var] < lower_bound) | (df[var] > upper_bound)]
    
    # Método Z-Score
    z_scores = np.abs((df[var] - df[var].mean()) / df[var].std())
    outliers_zscore[var] = df[z_scores > 3]
    
    print(f"\n📊 {var}:")
    print(f"   → Outliers (IQR): {len(outliers_iqr[var])} ({len(outliers_iqr[var])/len(df)*100:.1f}%)")
    print(f"   → Outliers (Z-Score): {len(outliers_zscore[var])} ({len(outliers_zscore[var])/len(df)*100:.1f}%)")
    print(f"   → Limites IQR: {lower_bound:.2f} a {upper_bound:.2f}")

print("✅ Análise de outliers concluída!")

🔍 Iniciando análise de outliers...

📊 Precipitation (mm day-1):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 1680.35 a 3340.72

📊 Specific Humidity at 2 Meters (g/kg):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 17.48 a 18.95

📊 Relative Humidity at 2 Meters (%):
   → Outliers (IQR): 0 (0.0%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 82.03 a 87.59

📊 Temperature at 2 Meters (C):
   → Outliers (IQR): 12 (7.7%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: 25.60 a 26.72

📊 Yield:
   → Outliers (IQR): 35 (22.4%)
   → Outliers (Z-Score): 0 (0.0%)
   → Limites IQR: -80458.75 a 156305.25
✅ Análise de outliers concluída!


In [8]:
# =============================================================================
# RESULTADOS DA ANÁLISE DE OUTLIERS
# =============================================================================

print("\n" + "="*60)
print("📊 RESULTADOS DA ANÁLISE DE OUTLIERS")
print("="*60)

print("🔍 COMPARAÇÃO DOS MÉTODOS:")
print("   → Precipitation: 0 outliers (ambos os métodos)")
print("   → Specific Humidity: 0 outliers (ambos os métodos)")
print("   → Relative Humidity: 0 outliers (ambos os métodos)")
print("   → Temperature: 12 outliers (IQR) vs 0 outliers (Z-Score)")
print("   → Yield: 35 outliers (IQR) vs 0 outliers (Z-Score)")

print("\n📊 ANÁLISE CRÍTICA:")
print("   ⚠️ Discordância entre métodos para Temperature e Yield")
print("   → IQR: Muito conservador, range muito estreito")
print("   → Z-Score: Mais realista, baseado em distribuição normal")
print("   → Limite negativo em Yield: Matematicamente impossível")

print("\n🌾 CONTEXTO AGRÍCOLA:")
print("   ✅ Yield bimodal: Característica natural (Oil palm vs outros)")
print("   ✅ Temperature: Variação normal para agricultura tropical")
print("   ✅ Variações ambientais: Dentro de ranges esperados")

print("\n💡 DECISÃO ESTRATÉGICA:")
print("   → NÃO remover outliers baseado em Z-Score (0 outliers)")
print("   → Manter todos os dados como informação valiosa")
print("   → Focar em feature engineering para capturar padrões")

print("✅ Resultados da análise de outliers concluídos!")


📊 RESULTADOS DA ANÁLISE DE OUTLIERS
🔍 COMPARAÇÃO DOS MÉTODOS:
   → Precipitation: 0 outliers (ambos os métodos)
   → Specific Humidity: 0 outliers (ambos os métodos)
   → Relative Humidity: 0 outliers (ambos os métodos)
   → Temperature: 12 outliers (IQR) vs 0 outliers (Z-Score)
   → Yield: 35 outliers (IQR) vs 0 outliers (Z-Score)

📊 ANÁLISE CRÍTICA:
   ⚠️ Discordância entre métodos para Temperature e Yield
   → IQR: Muito conservador, range muito estreito
   → Z-Score: Mais realista, baseado em distribuição normal
   → Limite negativo em Yield: Matematicamente impossível

🌾 CONTEXTO AGRÍCOLA:
   ✅ Yield bimodal: Característica natural (Oil palm vs outros)
   ✅ Temperature: Variação normal para agricultura tropical
   ✅ Variações ambientais: Dentro de ranges esperados

💡 DECISÃO ESTRATÉGICA:
   → NÃO remover outliers baseado em Z-Score (0 outliers)
   → Manter todos os dados como informação valiosa
   → Focar em feature engineering para capturar padrões
✅ Resultados da análise de out

## 2.3 🔧 Feature Engineering

### Objetivo da Seção
Criar novas variáveis derivadas baseadas em conhecimento do domínio agrícola:
- **Interações ambientais**: Relacionamentos entre variáveis climáticas
- **Índices climáticos**: Combinações com significado agrícola
- **Transformações**: Capturar relações não lineares
- **Base**: Features prontas para algoritmos de ML

### Estratégia de Criação:
- **Interações**: Temperature × Humidity, Precipitation × Temperature
- **Índices**: Conforto térmico, umidade efetiva
- **Transformações**: Logaritmos para distribuições assimétricas
- **Polinômios**: Capturar relações quadráticas

In [9]:
# =============================================================================
# FEATURE ENGINEERING - CRIAÇÃO DE VARIÁVEIS DERIVADAS
# =============================================================================

print("🔧 Iniciando Feature Engineering...")

# Backup dos dados originais
df_original = df.copy()

# 1. INTERAÇÕES AMBIENTAIS
print("\n📊 1. Criando interações ambientais...")

# Temperature × Relative Humidity (interação climática)
df['temp_humidity_interaction'] = df['Temperature at 2 Meters (C)'] * df['Relative Humidity at 2 Meters (%)']

# Precipitation × Temperature (condições de crescimento)
df['precip_temp_ratio'] = df['Precipitation (mm day-1)'] / (df['Temperature at 2 Meters (C)'] + 1)

# Specific × Relative Humidity (umidade combinada)
df['humidity_combined'] = df['Specific Humidity at 2 Meters (g/kg)'] * df['Relative Humidity at 2 Meters (%)'] / 100

print("   ✅ Interações ambientais criadas")

# 2. ÍNDICES CLIMÁTICOS
print("\n🌡️ 2. Criando índices climáticos...")

# Índice de conforto térmico
df['thermal_comfort'] = 0.5 * (df['Temperature at 2 Meters (C)'] + df['Relative Humidity at 2 Meters (%)'])

# Índice de umidade efetiva
df['effective_humidity'] = df['Specific Humidity at 2 Meters (g/kg)'] * df['Relative Humidity at 2 Meters (%)'] / 100

# Índice de condições de crescimento
df['growth_conditions'] = (df['Precipitation (mm day-1)'] * df['Relative Humidity at 2 Meters (%)']) / (df['Temperature at 2 Meters (C)'] + 1)

print("   ✅ Índices climáticos criados")

# 3. TRANSFORMAÇÕES
print("\n📈 3. Criando transformações...")

# Logaritmo para distribuições assimétricas
df['log_precipitation'] = np.log(df['Precipitation (mm day-1)'] + 1)

# Polinômios para relações não lineares
df['temperature_squared'] = df['Temperature at 2 Meters (C)'] ** 2
df['humidity_squared'] = df['Relative Humidity at 2 Meters (%)'] ** 2

print("   ✅ Transformações criadas")

# 4. RESUMO DAS FEATURES CRIADAS
print("\n📋 RESUMO DAS FEATURES:")
print(f"   → Features originais: {len(df_original.columns)}")
print(f"   → Features criadas: {len(df.columns) - len(df_original.columns)}")
print(f"   → Total de features: {len(df.columns)}")

# Listar novas features
new_features = [col for col in df.columns if col not in df_original.columns]
print(f"\n🆕 Novas features criadas:")
for i, feature in enumerate(new_features, 1):
    print(f"   {i}. {feature}")

print("✅ Feature Engineering concluído!")

🔧 Iniciando Feature Engineering...

📊 1. Criando interações ambientais...
   ✅ Interações ambientais criadas

🌡️ 2. Criando índices climáticos...
   ✅ Índices climáticos criados

📈 3. Criando transformações...
   ✅ Transformações criadas

📋 RESUMO DAS FEATURES:
   → Features originais: 6
   → Features criadas: 9
   → Total de features: 15

🆕 Novas features criadas:
   1. temp_humidity_interaction
   2. precip_temp_ratio
   3. humidity_combined
   4. thermal_comfort
   5. effective_humidity
   6. growth_conditions
   7. log_precipitation
   8. temperature_squared
   9. humidity_squared
✅ Feature Engineering concluído!


In [10]:
# =============================================================================
# RESULTADOS DO FEATURE ENGINEERING
# =============================================================================

print("\n" + "="*60)
print("🔧 RESULTADOS DO FEATURE ENGINEERING")
print("="*60)

print("📊 FEATURES CRIADAS:")
print(f"   → Features originais: {len(df_original.columns)}")
print(f"   → Features criadas: {len(df.columns) - len(df_original.columns)}")
print(f"   → Total de features: {len(df.columns)}")

print("\n🆕 NOVAS FEATURES:")
new_features = [col for col in df.columns if col not in df_original.columns]
for i, feature in enumerate(new_features, 1):
    print(f"   {i}. {feature}")

print("\n📈 ESTATÍSTICAS DAS NOVAS FEATURES:")
for feature in new_features:
    print(f"   → {feature}:")
    print(f"     Média: {df[feature].mean():.2f}")
    print(f"     Desvio: {df[feature].std():.2f}")
    print(f"     Min: {df[feature].min():.2f}")
    print(f"     Max: {df[feature].max():.2f}")

print("✅ Resultados do Feature Engineering concluídos!")


🔧 RESULTADOS DO FEATURE ENGINEERING
📊 FEATURES CRIADAS:
   → Features originais: 6
   → Features criadas: 9
   → Total de features: 15

🆕 NOVAS FEATURES:
   1. temp_humidity_interaction
   2. precip_temp_ratio
   3. humidity_combined
   4. thermal_comfort
   5. effective_humidity
   6. growth_conditions
   7. log_precipitation
   8. temperature_squared
   9. humidity_squared

📈 ESTATÍSTICAS DAS NOVAS FEATURES:
   → temp_humidity_interaction:
     Média: 2218.65
     Desvio: 27.86
     Min: 2143.89
     Max: 2261.80
   → precip_temp_ratio:
     Média: 91.49
     Desvio: 10.76
     Min: 71.10
     Max: 114.16
   → humidity_combined:
     Média: 15.43
     Desvio: 0.36
     Min: 14.40
     Max: 15.96
   → thermal_comfort:
     Média: 55.46
     Desvio: 0.47
     Min: 54.11
     Max: 56.14
   → effective_humidity:
     Média: 15.43
     Desvio: 0.36
     Min: 14.40
     Max: 15.96
   → growth_conditions:
     Média: 7760.61
     Desvio: 982.79
     Min: 5871.03
     Max: 9829.32
   → log_

## 2.4 📝 Encoding de Variáveis

### Objetivo da Seção
Transformar variáveis categóricas em formato numérico para algoritmos de ML:
- **One-Hot Encoding**: Criar colunas binárias para cada categoria
- **Label Encoding**: Transformar categorias em números
- **Preparação**: Variável 'Crop' pronta para modelagem

### Estratégia de Encoding:
- **One-Hot Encoding**: Para variável 'Crop' (4 categorias)
- **Verificação**: Confirmar transformação correta
- **Validação**: Garantir que não há vazamento de dados

In [11]:
# =============================================================================
# ENCODING DE VARIÁVEIS CATEGÓRICAS
# =============================================================================

print("�� Iniciando Encoding de variáveis...")

# Verificar variáveis categóricas
categorical_vars = df.select_dtypes(include=['object']).columns
print(f"📋 Variáveis categóricas encontradas: {list(categorical_vars)}")

# One-Hot Encoding para variável 'Crop'
print("\n🌾 Aplicando One-Hot Encoding para 'Crop'...")

# Criar dummies
crop_dummies = pd.get_dummies(df['Crop'], prefix='Crop')

# Mostrar as novas colunas criadas
print(f"🆕 Colunas criadas: {list(crop_dummies.columns)}")

# Verificar distribuição
print("\n📊 Distribuição das categorias:")
for col in crop_dummies.columns:
    count = crop_dummies[col].sum()
    percentage = (count / len(df)) * 100
    print(f"   → {col}: {count} registros ({percentage:.1f}%)")

# Adicionar ao dataset
df = pd.concat([df, crop_dummies], axis=1)

# Remover coluna original 'Crop'
df = df.drop('Crop', axis=1)

print(f"\n✅ Encoding concluído!")
print(f"�� Dataset final: {df.shape[0]} registros, {df.shape[1]} features")

�� Iniciando Encoding de variáveis...
📋 Variáveis categóricas encontradas: ['Crop']

🌾 Aplicando One-Hot Encoding para 'Crop'...
🆕 Colunas criadas: ['Crop_Cocoa, beans', 'Crop_Oil palm fruit', 'Crop_Rice, paddy', 'Crop_Rubber, natural']

📊 Distribuição das categorias:
   → Crop_Cocoa, beans: 39 registros (25.0%)
   → Crop_Oil palm fruit: 39 registros (25.0%)
   → Crop_Rice, paddy: 39 registros (25.0%)
   → Crop_Rubber, natural: 39 registros (25.0%)

✅ Encoding concluído!
�� Dataset final: 156 registros, 18 features


## 2.5 🔬 Normalização de Variáveis

### Objetivo da Seção
Padronizar variáveis numéricas para melhorar performance de algoritmos de ML:
- **StandardScaler**: Normalizar para média=0 e desvio=1
- **Balanceamento**: Evitar que uma variável domine o modelo
- **Preparação**: Dados prontos para algoritmos sensíveis à escala

### Estratégia de Normalização:
- **Seleção**: Apenas variáveis numéricas (excluindo dummies)
- **StandardScaler**: Método mais comum e eficaz
- **Verificação**: Confirmar normalização correta
- **Preservação**: Manter estrutura dos dados

In [20]:
# =============================================================================
# NORMALIZAÇÃO DE VARIÁVEIS NUMÉRICAS
# =============================================================================

print("🔬 Iniciando Normalização...")

# Selecionar variáveis numéricas (excluindo dummies E YIELD)
numeric_features = df.select_dtypes(include=[np.number]).columns
dummy_features = [col for col in df.columns if col.startswith('Crop_')]
features_to_scale = [col for col in numeric_features if col not in dummy_features and col != 'Yield']

print(f"📊 Variáveis para normalizar: {len(features_to_scale)}")
print(f"📋 Features: {list(features_to_scale)}")

# Backup dos dados originais
df_scaled = df.copy()

# Aplicar StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Normalizar features selecionadas
df_scaled[features_to_scale] = scaler.fit_transform(df[features_to_scale])

# Verificar normalização
print("\n📈 VERIFICAÇÃO DA NORMALIZAÇÃO:")
for feature in features_to_scale:
    mean_val = df_scaled[feature].mean()
    std_val = df_scaled[feature].std()
    print(f"   → {feature}:")
    print(f"     Média: {mean_val:.6f} (deveria ser ~0)")
    print(f"     Desvio: {std_val:.6f} (deveria ser ~1)")

# Verificar que Yield não foi normalizado
print(f"\n✅ Yield mantido original: {df_scaled['Yield'].mean():.2f}")

# Salvar scaler para uso futuro
import joblib
joblib.dump(scaler, '../models/scaler.pkl')

print(f"\n✅ Normalização concluída!")
print(f"📊 Dataset normalizado: {df_scaled.shape[0]} registros, {df_scaled.shape[1]} features")
print(f"💾 Scaler salvo em: ../models/scaler.pkl")

🔬 Iniciando Normalização...
📊 Variáveis para normalizar: 13
📋 Features: ['Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)', 'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)', 'temp_humidity_interaction', 'precip_temp_ratio', 'humidity_combined', 'thermal_comfort', 'effective_humidity', 'growth_conditions', 'log_precipitation', 'temperature_squared', 'humidity_squared']

📈 VERIFICAÇÃO DA NORMALIZAÇÃO:
   → Precipitation (mm day-1):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Specific Humidity at 2 Meters (g/kg):
     Média: -0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Relative Humidity at 2 Meters (%):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → Temperature at 2 Meters (C):
     Média: 0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser ~1)
   → temp_humidity_interaction:
     Média: -0.000000 (deveria ser ~0)
     Desvio: 1.003221 (deveria ser

## 2.6 ✅ Dados Finais

### Objetivo da Seção
Verificar e documentar o dataset final preparado para modelagem:
- **Estrutura final**: Confirmar todas as transformações
- **Qualidade dos dados**: Verificar integridade e consistência
- **Preparação**: Dataset pronto para algoritmos de ML
- **Documentação**: Resumo das transformações aplicadas

### Estratégia de Verificação:
- **Resumo estatístico**: Estatísticas descritivas finais
- **Verificação de tipos**: Confirmar tipos de dados corretos
- **Verificação de valores**: Confirmar ranges e distribuições
- **Salvamento**: Salvar dataset final para próximas etapas

In [18]:
# =============================================================================
# DADOS FINAIS - VERIFICAÇÃO E SALVAMENTO
# =============================================================================

print("✅ Verificando dados finais...")

# Resumo final do dataset
print(f"📊 ESTRUTURA FINAL:")
print(f"   → Registros: {df_scaled.shape[0]}")
print(f"   → Features: {df_scaled.shape[1]}")
print(f"   → Features normalizadas: {len(features_to_scale)}")
print(f"   → Features categóricas (dummies): {len(dummy_features)}")
print(f"   → Variável alvo: Yield (original)")

# Verificar tipos de dados
print(f"\n�� TIPOS DE DADOS:")
print(df_scaled.dtypes.value_counts())

# Resumo estatístico das features normalizadas
print(f"\n📈 ESTATÍSTICAS DAS FEATURES NORMALIZADAS:")
print(df_scaled[features_to_scale].describe())

# Verificar Yield (não normalizado)
print(f"\n�� YIELD (VARIÁVEL ALVO):")
print(f"   → Média: {df_scaled['Yield'].mean():.2f}")
print(f"   → Desvio: {df_scaled['Yield'].std():.2f}")
print(f"   → Min: {df_scaled['Yield'].min():.2f}")
print(f"   → Max: {df_scaled['Yield'].max():.2f}")

# Salvar dataset final
df_scaled.to_csv('../data/processed/dataset_ready.csv', index=False)

print(f"\n💾 Dataset salvo em: ../data/processed/dataset_ready.csv")
print("✅ Preparação de dados concluída!")

✅ Verificando dados finais...
📊 ESTRUTURA FINAL:
   → Registros: 156
   → Features: 18
   → Features normalizadas: 13
   → Features categóricas (dummies): 4
   → Variável alvo: Yield (original)

�� TIPOS DE DADOS:
float64    13
bool        4
int64       1
Name: count, dtype: int64

📈 ESTATÍSTICAS DAS FEATURES NORMALIZADAS:
       Precipitation (mm day-1)  Specific Humidity at 2 Meters (g/kg)  \
count              1.560000e+02                          1.560000e+02   
mean               1.231921e-15                         -5.277829e-15   
std                1.003221e+00                          1.003221e+00   
min               -1.912735e+00                         -2.263223e+00   
25%               -6.360164e-01                         -5.907486e-01   
50%               -2.147065e-01                          2.284228e-01   
75%                8.026274e-01                          6.721406e-01   
max                2.077059e+00                          1.696105e+00   

       Relative H

In [19]:
# =============================================================================
# RESULTADOS DA PREPARAÇÃO DE DADOS
# =============================================================================

print("\n" + "="*60)
print("🔧 RESULTADOS DA PREPARAÇÃO DE DADOS")
print("="*60)

print("📊 TRANSFORMAÇÕES APLICADAS:")
print("   ✅ Análise de outliers: 0 outliers reais detectados")
print("   ✅ Feature Engineering: 9 novas features criadas")
print("   ✅ One-Hot Encoding: Variável 'Crop' transformada em 4 dummies")
print("   ✅ Normalização: 13 features padronizadas (média=0, desvio=1)")

print("\n📈 FEATURES CRIADAS:")
print("   → Interações ambientais: temp_humidity_interaction, precip_temp_ratio")
print("   → Índices climáticos: thermal_comfort, effective_humidity, growth_conditions")
print("   → Transformações: log_precipitation, temperature_squared, humidity_squared")

print("\n📋 ESTRUTURA FINAL:")
print("   → Registros: 156 (todos preservados)")
print("   → Features: 18 (5 originais + 9 criadas + 4 dummies)")
print("   → Features normalizadas: 13 (prontas para ML)")
print("   → Variável alvo: Yield (escala original preservada)")

print("✅ Resultados da preparação de dados concluídos!")


🔧 RESULTADOS DA PREPARAÇÃO DE DADOS
📊 TRANSFORMAÇÕES APLICADAS:
   ✅ Análise de outliers: 0 outliers reais detectados
   ✅ Feature Engineering: 9 novas features criadas
   ✅ One-Hot Encoding: Variável 'Crop' transformada em 4 dummies
   ✅ Normalização: 13 features padronizadas (média=0, desvio=1)

📈 FEATURES CRIADAS:
   → Interações ambientais: temp_humidity_interaction, precip_temp_ratio
   → Índices climáticos: thermal_comfort, effective_humidity, growth_conditions
   → Transformações: log_precipitation, temperature_squared, humidity_squared

📋 ESTRUTURA FINAL:
   → Registros: 156 (todos preservados)
   → Features: 18 (5 originais + 9 criadas + 4 dummies)
   → Features normalizadas: 13 (prontas para ML)
   → Variável alvo: Yield (escala original preservada)
✅ Resultados da preparação de dados concluídos!


## 📋 Conclusões da Preparação de Dados

### 🔍 Análise de Outliers
- **Z-Score**: 0 outliers reais detectados
- **IQR**: Falsos positivos em Temperature e Yield
- **Decisão**: Manter todos os dados como informação valiosa

### 🔧 Feature Engineering
- **9 features criadas** baseadas em domínio agrícola
- **Interações**: Temperature × Humidity, Precipitation × Temperature
- **Índices**: Conforto térmico, umidade efetiva, condições de crescimento
- **Transformações**: Log para precipitação, polinômios para relações não lineares

### 📝 Encoding
- **One-Hot Encoding** aplicado à variável 'Crop'
- **4 colunas binárias** criadas: Crop_Cocoa, Crop_OilPalm, Crop_Rice, Crop_Rubber
- **Sem hierarquia artificial** entre culturas

### 🔬 Normalização
- **StandardScaler** aplicado a 13 features numéricas
- **Todas as features**: Média ≈ 0, Desvio ≈ 1
- **Yield mantido** na escala original (variável alvo)
- **Scaler salvo** para uso futuro

### 💡 Insights para Modelagem
- Dataset bem estruturado para algoritmos de ML
- Features com significado agrícola claro
- Normalização adequada para algoritmos sensíveis à escala
- Variável alvo preservada para interpretação correta

### 🎯 Próximos Passos
- **Clustering**: Análise de tendências de produtividade
- **Modelagem**: 5 algoritmos diferentes
- **Avaliação**: Comparação de performance